# IBM Data Science Capstone Course

### Opening a new Bakery in Mumbai, India.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print("Libraries imported.")

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                       

In [2]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_of_Mumbai").text

In [3]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [4]:
# create a list to store neighborhood data
neighborhoodList = []

In [5]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [6]:
# create a new DataFrame from the list
mum_df = pd.DataFrame({"Neighborhood": neighborhoodList})

mum_df.head()

,Neighborhood
0,Andheri
1,Anushakti Nagar
2,Baiganwadi
3,Bandra
4,Bhandup


In [7]:
mum_df.shape

(42, 1)

In [8]:
!pip install geocoder
import geocoder

def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

     |████████████████████████████████| 102kB 6.9MB/s ta 0:00:011


In [9]:
coords = [ get_latlng(neighborhood) for neighborhood in mum_df["Neighborhood"].tolist() ]

In [10]:
coords

[[19.11846999503601, 72.8417699936972],
 [19.042830000000038, 72.92734000000007],
 [19.062940000000026, 72.92663000000005],
 [19.054370000000063, 72.84017000000006],
 [19.145560000000046, 72.94856000000004],
 [19.229360000000042, 72.85751000000005],
 [19.208660000000066, 72.82612000000006],
 [19.053995033751242, 72.89967511449926],
 [19.250030000000038, 72.85907000000003],
 [19.224690000000066, 72.86605000000003],
 [19.212750000000028, 73.08324000000005],
 [19.00427193812948, 72.85579021275656],
 [19.086299999191567, 72.90908002895725],
 [19.164550000000077, 72.84946000000008],
 [18.959290000000067, 72.83108000000004],
 [19.137920000000065, 72.84941000000003],
 [19.014920000000075, 72.84522000000004],
 [18.953939983134134, 72.8203699890619],
 [19.211900010338013, 72.83750000076253],
 [19.131380000000036, 72.93568000000005],
 [19.127580000000023, 72.82539000000008],
 [19.064980000000048, 72.88069000000007],
 [19.21094000000005, 72.84137000000004],
 [19.048530000000028, 72.93222000000003

In [11]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [12]:
# merge the coordinates into the original dataframe
mum_df['Latitude'] = df_coords['Latitude']
mum_df['Longitude'] = df_coords['Longitude']

In [13]:
print(mum_df.shape)
mum_df

(42, 3)


,Neighborhood,Latitude,Longitude
0,Andheri,19.118470,72.841770
1,Anushakti Nagar,19.042830,72.927340
2,Baiganwadi,19.062940,72.926630
3,Bandra,19.054370,72.840170
4,Bhandup,19.145560,72.948560
5,Borivali,19.229360,72.857510
6,Charkop,19.208660,72.826120
7,Chembur,19.053995,72.899675
8,Dahisar,19.250030,72.859070
9,Devipada,19.224690,72.866050


In [14]:
mum_df.to_csv("mum_df.csv", index=False)

In [15]:
# get the coordinates of Mumbai
address = 'Mumbai, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai, India 19.0759899, 72.8773928.


In [16]:
map_mum = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(mum_df['Latitude'], mum_df['Longitude'], mum_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_mum)  
    
map_mum

In [17]:
map_mum.save('map_mum.html')

In [18]:
# define Foursquare Credentials and Version
CLIENT_ID = 'NOV52PLAXIPSXEHJS0W0FYVHL1CCOVFAEMABY2DDDX55WYCI'
CLIENT_SECRET = 'CQYWIBSK4I0B2DK4LJJRADJLUTR5NHCXL4E2XN3RLJVLZXFU'
VERSION = '20200716' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NOV52PLAXIPSXEHJS0W0FYVHL1CCOVFAEMABY2DDDX55WYCI
CLIENT_SECRET:CQYWIBSK4I0B2DK4LJJRADJLUTR5NHCXL4E2XN3RLJVLZXFU


In [19]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(mum_df['Latitude'], mum_df['Longitude'], mum_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [20]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(2694, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Andheri,19.11847,72.84177,Merwans Cake shop,19.119300,72.845418,Bakery
1,Andheri,19.11847,72.84177,Radha Krishna Veg Restaurant,19.115130,72.843060,Indian Restaurant
2,Andheri,19.11847,72.84177,Shawarma Factory,19.124591,72.840398,Falafel Restaurant
3,Andheri,19.11847,72.84177,Naturals,19.111204,72.837255,Ice Cream Shop
4,Andheri,19.11847,72.84177,Temptations,19.113767,72.841337,Ice Cream Shop


In [21]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Andheri,100,100,100,100,100,100
Anushakti Nagar,13,13,13,13,13,13
Baiganwadi,8,8,8,8,8,8
Bandra,100,100,100,100,100,100
Bhandup,25,25,25,25,25,25
Borivali,100,100,100,100,100,100
Charkop,51,51,51,51,51,51
Chembur,81,81,81,81,81,81
Dahisar,60,60,60,60,60,60


In [22]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))


There are 199 uniques categories.


In [23]:
venues_df['VenueCategory'].unique()[:50]

array(['Bakery', 'Indian Restaurant', 'Falafel Restaurant',
       'Ice Cream Shop', 'Pub', 'Coffee Shop', 'Sandwich Place',
       'Pizza Place', 'Multiplex', 'Breakfast Spot', 'Juice Bar', 'Café',
       'Department Store', 'American Restaurant', 'Cocktail Bar', 'Diner',
       'Snack Place', 'Chinese Restaurant', 'Seafood Restaurant',
       'Maharashtrian Restaurant', 'Bar', "Women's Store", 'Lounge',
       'Residential Building (Apartment / Condo)', 'Fast Food Restaurant',
       'Gym / Fitness Center', 'Spa', 'Electronics Store',
       'Asian Restaurant', 'Smoke Shop', 'Vegetarian / Vegan Restaurant',
       'Food Truck', 'Liquor Store', 'Fish Market', 'Park', 'Tea Room',
       'Martial Arts Dojo', 'Athletics & Sports', 'Dessert Shop', 'Food',
       'Plaza', 'Platform', 'Paper / Office Supplies Store',
       'Food & Drink Shop', 'Music Venue', 'Hot Dog Joint',
       'Fried Chicken Joint', 'Gym', 'Gourmet Shop', 'Sports Club'],
      dtype=object)

In [24]:
# check if the results contain "Restaurent"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

In [25]:
# one hot encoding
mum_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mum_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [mum_onehot.columns[-1]] + list(mum_onehot.columns[:-1])
mum_onehot = mum_onehot[fixed_columns]

print(mum_onehot.shape)
mum_onehot.head()

(2694, 200)


,Neighborhoods,ATM,Afghan Restaurant,American Restaurant,Antique Shop,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Garden,Big Box Store,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Cafeteria,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gujarati Restaurant,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Lake,Light Rail Station,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Motorcycle Shop,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Social Club,South American Restaurant,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Trail,Train,Train Station,Travel & Transport,Travel Lounge,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Andheri,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [26]:
mum_grouped = mum_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(mum_grouped.shape)
mum_grouped

(41, 200)


,Neighborhoods,ATM,Afghan Restaurant,American Restaurant,Antique Shop,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Garden,Big Box Store,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Cafeteria,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gujarati Restaurant,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Lake,Light Rail Station,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Motorcycle Shop,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Social Club,South American Restaurant,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Trail,Train,Train Station,Travel & Transport,Travel Lounge,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Andheri,0.0,0.000000,0.010000,0.00,0.000000,0.000000,0.020000,0.010000,0.000000,0.00,0.020000,0.000000,0.060000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.040000,0.00,0.050000,0.000000,0.030000,0.020000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.00,0.010000,0.00,0.000000,0.020000,0.000000,0.010000,0.000000,0.000000,0.010000,0.00,0.000000,0.020000,0.00,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.140000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.010000,0.020000,0.010000,0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00

In [27]:
len(mum_grouped[mum_grouped["Bakery"] > 0])

27

In [28]:
mum_bakery = mum_grouped[["Neighborhoods","Bakery"]]

In [29]:
mum_bakery.head()

,Neighborhoods,Bakery
0,Andheri,0.02
1,Anushakti Nagar,0.00
2,Baiganwadi,0.00
3,Bandra,0.07
4,Bhandup,0.00


In [31]:
kclusters = 5

mum_clustering = mum_bakery.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mum_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 3, 3, 2, 3, 0, 0, 3, 0, 3], dtype=int32)

In [32]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
mum_merged = mum_bakery.copy()

# add clustering labels
mum_merged["Cluster Labels"] = kmeans.labels_

In [33]:
mum_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
mum_merged.head()

,Neighborhood,Bakery,Cluster Labels
0,Andheri,0.02,0
1,Anushakti Nagar,0.00,3
2,Baiganwadi,0.00,3
3,Bandra,0.07,2
4,Bhandup,0.00,3


In [34]:
# merge mum_grouped with mum_data to add latitude/longitude for each neighborhood
mum_merged = mum_merged.join(mum_df.set_index("Neighborhood"), on="Neighborhood")

print(mum_merged.shape)
mum_merged.head() # check the last columns!

(41, 5)


,Neighborhood,Bakery,Cluster Labels,Latitude,Longitude
0,Andheri,0.02,0,19.11847,72.84177
1,Anushakti Nagar,0.00,3,19.04283,72.92734
2,Baiganwadi,0.00,3,19.06294,72.92663
3,Bandra,0.07,2,19.05437,72.84017
4,Bhandup,0.00,3,19.14556,72.94856


In [35]:
# sort the results by Cluster Labels
print(mum_merged.shape)
mum_merged.sort_values(["Cluster Labels"], inplace=True)
mum_merged

(41, 5)


,Neighborhood,Bakery,Cluster Labels,Latitude,Longitude
0,Andheri,0.020000,0,19.118470,72.841770
29,Seven Bungalows,0.040000,0,19.131460,72.816460
24,"Matharpacady, Mumbai",0.030000,0,18.950710,72.827270
22,Mahavir Nagar (Kandivali),0.025641,0,19.210940,72.841370
21,Kurla,0.021277,0,19.064980,72.880690
18,Kandivali,0.025000,0,19.211900,72.837500
17,Kalyan,0.030000,0,18.953940,72.820370
16,Juhu,0.040000,0,19.014920,72.845220
14,Grant Road,0.040000,0,18.959290,72.831080
13,Goregaon,0.020000,0,19.164550,72.849460


In [36]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mum_merged['Latitude'], mum_merged['Longitude'], mum_merged['Neighborhood'], mum_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [37]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

In [38]:
# cluster 0
mum_merged.loc[mum_merged['Cluster Labels'] == 0]

,Neighborhood,Bakery,Cluster Labels,Latitude,Longitude
0,Andheri,0.020000,0,19.11847,72.84177
29,Seven Bungalows,0.040000,0,19.13146,72.81646
24,"Matharpacady, Mumbai",0.030000,0,18.95071,72.82727
22,Mahavir Nagar (Kandivali),0.025641,0,19.21094,72.84137
21,Kurla,0.021277,0,19.06498,72.88069
18,Kandivali,0.025000,0,19.21190,72.83750
17,Kalyan,0.030000,0,18.95394,72.82037
16,Juhu,0.040000,0,19.01492,72.84522
14,Grant Road,0.040000,0,18.95929,72.83108
13,Goregaon,0.020000,0,19.16455,72.84946


In [39]:
# cluster 1
mum_merged.loc[mum_merged['Cluster Labels'] == 1]

,Neighborhood,Bakery,Cluster Labels,Latitude,Longitude
27,Mumbra,0.25,1,19.188413,73.022011


In [40]:
# cluster 2
mum_merged.loc[mum_merged['Cluster Labels'] == 2]

,Neighborhood,Bakery,Cluster Labels,Latitude,Longitude
10,Dombivli,0.052632,2,19.21275,73.08324
3,Bandra,0.070000,2,19.05437,72.84017
40,Worli,0.070000,2,19.00744,72.81688
34,Tilak Nagar (Mumbai),0.047619,2,18.99616,72.85279


In [41]:
# cluster 3
mum_merged.loc[mum_merged['Cluster Labels'] == 3]

,Neighborhood,Bakery,Cluster Labels,Latitude,Longitude
36,Vashi,0.010638,3,19.080060,72.998610
28,Pestom sagar,0.012346,3,19.070640,72.902170
32,"Sonapur, Bhandup",0.000000,3,19.163940,72.935440
31,"Sion, Mumbai",0.012821,3,19.043410,72.863320
37,Vikhroli,0.010000,3,19.111090,72.927810
30,Shil Phata,0.000000,3,19.146580,73.040050
1,Anushakti Nagar,0.000000,3,19.042830,72.927340
38,Wadala,0.000000,3,19.017200,72.858170
33,Thakur village,0.000000,3,19.210200,72.875410
11,Eastern Suburbs (Mumbai),0.000000,3,19.004272,72.855790


In [42]:
# cluster 4
mum_merged.loc[mum_merged['Cluster Labels'] == 4]

,Neighborhood,Bakery,Cluster Labels,Latitude,Longitude
35,Uttan,0.190476,4,26.86634,80.93884


# Observation

By results of clustering we can say that cluster 0 and cluster 3 have hogest no of Bakeries, cluster 2 has moderate and cluster 1 and 4 have only one bakeries in neighborhoods. So clusters 1, 2, 3 have high concentrationn than other areas. So in Mumbai, India the areas Uttan and Mumbra are good to open new bakery.